In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from helper.modules import EpilepticSeizure, LeaveOneOut
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping

In [2]:
config = {
    "data_path": "Data/",
    "preprocess": True,
    "generate_images": True,
    "rows_per_person": 23,
    "frequency": 178,
    "labels": 2,
    "add_noise": True,
    "oversampling": True,
    "undersampling": True,
    "test_size": 0.3
}

es = EpilepticSeizure(config)

X_train, y_train, X_test, y_test = es.process()

X_train.shape, y_train.shape, X_test.shape, y_test.shape

Processing Patients Test: 100%|██████████| 150/150 [00:03<00:00, 39.71it/s]


((32200, 179), (32200, 1), (3450, 179), (3450, 1))

In [ ]:
def scheduler(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

def build_model(num_classes=1):
    tf.keras.backend.clear_session()
    model = Sequential()
    model.add(LSTM(256, input_shape=(1, 178), activation='relu', return_sequences=True))
    model.add(LSTM(128, activation='relu', return_sequences=True))
    model.add(LSTM(64, activation='relu', return_sequences=True))
    model.add(LSTM(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))

    lr_scheduler = LearningRateScheduler(scheduler)

    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
model = build_model()
model.summary()

In [ ]:
test = LeaveOneOut(data, build_model, 14, 'porn_addiction')

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='loss', patience=5, verbose=1)

models = test.train(epoch = 5, batch_size = 1250, callbacks = [early_stopping])